In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import numpy as np
import pandas as pd
import cv2
import torchvision.transforms as transforms

class MyDataset(torch.utils.data.Dataset):
    # データは data_path ディレクトリの中に入れておく．
    # data_label_file は以下のような csv ファイル（.csvという名前のテキストファイル）のファイル名．
    # 1行目は "img, label"，2行目以降は "画像ファイル名，ラベル"
    #
    # 下は data_label_file の例
    #
    # img, label
    # img1.png, 1
    # img2.png, 0
    # img3.png, 1

    def __init__(self, data_path, data_label_file, transform=None):
        self.data_path = data_path
        self.df = pd.read_csv(self.data_path + "/" + data_label_file)
        self.transform = transform

    # データの数を返す関数
    def __len__(self):
        return len(self.df)

    # i 番目のデータを取得するための関数
    def __getitem__(self, i):
        img_file = self.df['img'][i]
        print(img_file)
        img = torch.tensor(cv2.imread(self.data_path + "/" + img_file, cv2.IMREAD_GRAYSCALE))
        label = np.array(self.df['label'][i])
        if self.transform:
            img = self.transform(img.unsqueeze(0))
        return torch.tensor(img/255.0,dtype=torch.float), torch.tensor(label/250.0,dtype=torch.float)

# データを mydata に入れておき，データとラベルの対応付けのファイルを datalist*.csv とした場合
BATCH_SIZE = 1
transform = transforms.Resize([150,110])
train_data = MyDataset(data_path="drive/MyDrive/Colab Notebooks/dounuts",data_label_file="datalist.csv",transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_data = MyDataset(data_path="drive/MyDrive/Colab Notebooks/dounuts2",data_label_file="datalist2.csv",transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# pytorch 関係のライブラリの読み込み
import torch
import torch.nn as nn
import torch.optim as optim

# ニューラルネットワークの定義
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        # ネットワーク中の線形変換の定義
        self.l1 = nn.Linear(input_size,hidden_size)
        self.l2 = nn.Linear(hidden_size,hidden_size)
        self.l3 = nn.Linear(hidden_size,output_size)

    def forward(self, x):
        # ネットワーク中の具体的な計算
        x = self.l1(x)
        # 活性化関数．tanh, sigmoid, relu など
        #x = torch.tanh(x)
        #x = torch.sigmoid(x)
        x = torch.relu(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.l3(x)
        return x

In [ ]:
import random
# GPU を使って計算．何か不具合が生じた場合は CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'

# 使えるGPUの確認．
!nvidia-smi

# ネットワークを生成．
# ニューラルネットワークのサイズを決定．
num_inputs = 16500
num_outputs = 1
num_hidden = 16500
mynet = MLP(num_inputs,num_hidden,num_outputs).to(device)

# どれだけ長く学習するか．
num_epochs = 100

# 誤差の計算方法を指定．
criterion = nn.MSELoss()

# 学習アルゴリズムとパラメータを指定．
optimizer = optim.Adam(params=mynet.parameters(), lr=0.00001)

history_loss = []
history_eval = []
history_acc = []
# 実際の学習部分．
for epoch in range(num_epochs):
  # ネットワークを学習モードに切り替える．
  mynet.train()

  total_loss = 0.0
  mynet.load_state_dict(torch.load("drive/MyDrive/model.pth"))
  for i, (data, target) in enumerate(train_loader):
    # 微分をゼロに初期化．ネットワークの出力，損失関数を計算し，その微分を求め，学習を進める
    data = data.view(-1, num_inputs)
    optimizer.zero_grad()
    output = mynet(data.to(device))
    loss = criterion(output, target.to(device))
    loss.backward()
    optimizer.step()

    # 各バッチでの損失関数の値を合計．
    total_loss = total_loss + loss.cpu().item()
  #タイムアウトに備えてモデルを一旦保存する
  torch.save(mynet.state_dict(), "drive/MyDrive/model.pth")
  #前回の実行結果をロードする時
  #model.load_state_dict(torch.load("drive/MyDrive/model.pth"))
  # （学習データとは別の）テストデータで性能を検証．
  num_correct = 0
  num_data = 0
  # 性能を検証するときには学習する必要は無いので，ネットワークを評価モードにし，微分計算無しで計算．
  mynet.eval()
  with torch.no_grad():
    eval_loss = 0.0
    for i, (data, target) in enumerate(test_loader):
      data = data.view(-1, num_inputs)
      output = mynet(data.to(device))
      loss = criterion(output, target.to(device))
      print(250*output)
      print(250*target.to(device))
      eval_loss = total_loss + loss.cpu().item()
      num_correct = num_correct + output.cpu().eq(target).sum()
      num_data = num_data + data.shape[0]
  torch.save(mynet.state_dict(), "drive/MyDrive/model.pth")

  history_loss.append(total_loss)
  history_eval.append(eval_loss)
  history_acc.append(num_correct.item()/num_data)
  print("{}/{} training loss: {}, evaluation loss: {}".format(epoch,num_epochs,total_loss,eval_loss))